# Getting sentence representations

https://huggingface.co/transformers/pretrained_models.html

In [23]:
pretrained1="wietsedv/bert-base-dutch-cased"
pretrained2="pdelobelle/robbert-v2-dutch-base"

In [25]:
from simpletransformers.language_representation import RepresentationModel

model1 = RepresentationModel(
        model_type="bert",
        model_name=pretrained1,
        use_cuda=False
    )

model2 = RepresentationModel(
        model_type="roberta",
        model_name=pretrained2,
        use_cuda=False
    )

Some weights of the model checkpoint at wietsedv/bert-base-dutch-cased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing BertForTextRepresentation: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediate.den

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [26]:
from sentence_transformers import util
def get_cos_sim(sen1, sen2):
    cos_scores = util.pytorch_cos_sim(sen1, sen2)[0]
    cos_scores = cos_scores.cpu()
    return cos_scores

In [35]:
import pandas as pd
file='/Users/piek/Desktop/r-MedicalBertje/Samples/similarity-samples/similarity_examples4.xlsx'
df = pd.read_excel(file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sen1    40 non-null     object 
 1   Sen2    39 non-null     object 
 2   Sim     40 non-null     float64
 3   ID      40 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.4+ KB


In [38]:
scores = []
for index, row in df.iterrows():
    #print(row['Sen1'], row['Sen2'])
    sentences= [row['Sen1'], row['Sen2']]
    cos = -1
    try:
        word_vectors = model1.encode_sentences(sentences, combine_strategy="mean")
        assert word_vectors.shape == (2, 768)
        cos = get_cos_sim(word_vectors[0], word_vectors[1])
        scores.append(cos.item())
    except:
        print('reading error', sentences)
        scores.append(-1)
    
df[pretrained1]=scores

scores = []
for index, row in df.iterrows():
    #print(row['Sen1'], row['Sen2'])
    sentences= [row['Sen1'], row['Sen2']]
    
    cos = -1
    try:
        word_vectors = model1.encode_sentences(sentences, combine_strategy="mean")
        assert word_vectors.shape == (2, 768)
        cos = get_cos_sim(word_vectors[0], word_vectors[1])
        scores.append(cos.item())
    except:
        print('reading error', sentences)
        scores.append(-1)
    
df[pretrained2]=scores

df.to_excel(file)

reading error
reading error


In [6]:
from transformers import pipeline

pipe = pipeline('fill-mask', model='GroNLP/bert-base-dutch-cased')
for res in pipe('Meneer Janssen is gediagnosticeerd met [MASK].'):
    print(res['sequence'])

Some weights of BertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Meneer Janssen is gediagnosticeerd met kanker.
Meneer Janssen is gediagnosticeerd met.
Meneer Janssen is gediagnosticeerd met dementie.
Meneer Janssen is gediagnosticeerd met longkanker.
Meneer Janssen is gediagnosticeerd met borstkanker.
